In [ ]:
%load_ext autoreload
%autoreload 2

from lgsm.speculator import *
import torch
import matplotlib.pyplot as plt
import pickle
import numpy as np

In [ ]:
config = speculator_uv_config
path = "models/uv_model_tf.pkl"

model = SpeculatorModule(config)

with open(path, "rb") as file:
    tf_params = pickle.load(file)

model.latent_shift = nn.Parameter(torch.from_numpy(tf_params["parameters_shift"]))
model.latent_scale = nn.Parameter(torch.from_numpy(tf_params["parameters_scale"]))
for layer, W, b in zip(model.linear_layers[::2], tf_params["W"], tf_params["b"]):
    layer.weight = nn.Parameter(torch.from_numpy(W.T))
    layer.bias = nn.Parameter(torch.from_numpy(b))
for layer, alpha, beta in zip(model.linear_layers[1::2], tf_params["alphas"], tf_params["betas"]):
    layer.alpha = nn.Parameter(torch.from_numpy(alpha))
    layer.beta = nn.Parameter(torch.from_numpy(beta))
model.pca_shift = nn.Parameter(torch.from_numpy(tf_params["pca_shift"]))
model.pca_scale = nn.Parameter(torch.from_numpy(tf_params["pca_scale"]))
model.wavelengths = nn.Parameter(torch.from_numpy(tf_params["wavelengths"]))
model.pca_basis = nn.Parameter(torch.from_numpy(tf_params["pca_transform_matrix"]))
model.log_spectrum_shift = nn.Parameter(torch.from_numpy(tf_params["log_spectrum_shift"]))
model.log_spectrum_scale = nn.Parameter(torch.from_numpy(tf_params["log_spectrum_scale"]))

In [ ]:
torch.save(model.state_dict(), "models/speculator_uv_model_.pt")

In [ ]:
rng = np.random.default_rng(123)
theta = rng.normal(size=14) * 1e-9
latents = torch.from_numpy(theta).float()

plt.plot(model.wavelengths.detach(), model(latents).detach())